In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
freebase_url = "http://210.28.134.34:8890/sparql"
endpoint = SPARQLWrapper(freebase_url)
endpoint.setReturnFormat(JSON)

In [1]:
import pyodbc
def initialize_odbc_connection():
    global odbc_conn
    odbc_conn = pyodbc.connect(
        'DRIVER=/home3/xwu/virtuoso/virtodbc.so;Host=localhost:1111;UID=dba;PWD=dba'
    )
    odbc_conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
    odbc_conn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
    odbc_conn.setencoding(encoding='utf8')
    print('Freebase Virtuoso ODBC connected')

def query_with_odbc(query):
    try:
        with odbc_conn.cursor() as cursor:
            cursor.execute(query)
            rows = cursor.fetchall()
    except Exception as err:
        print(err)
        print(f"Query Execution Failed:{query}")
        initialize_odbc_connection()
        return []    
    return rows

initialize_odbc_connection()

Freebase Virtuoso ODBC connected


In [3]:
def convert_time2value(raw_time_str):
    if raw_time_str=="null":
        return raw_time_str
    bc=""
    res=raw_time_str
    if raw_time_str.startswith("-"):
        bc="-"
    if raw_time_str.find(':')!=-1:
        res=raw_time_str[:(raw_time_str.find(':')-2)].strip()
    res=res.replace("-","")
    if len(res)==4:
        res=res+"0101"
    return bc+res

spouse_dict={"r_name":"people.person.spouse_s","cvt_typ":"people.marriage","p2s":"people.marriage.spouse","p2o":"people.marriage.spouse","p2st":"people.marriage.from","p2en":"people.marriage.to"}
memofsportsteam_dict={"r_name":"sports.pro_athlete.teams","cvt_typ":"sports.sports_team_roster","p2s":"sports.sports_team_roster.player","p2o":"sports.sports_team_roster.team","p2st":"sports.sports_team_roster.from","p2en":"sports.sports_team_roster.to"}
employer_dict={"r_name":"people.person.employment_history","cvt_typ":"business.employment_tenure","p2s":"business.employment_tenure.person","p2o":"business.employment_tenure.company","p2st":"business.employment_tenure.from","p2en":"business.employment_tenure.to"}
coach_dict={"r_name":"sports.sports_team.coaches","cvt_typ":"sports.sports_team_coach_tenure","p2s":"sports.sports_team_coach_tenure.team","p2o":"sports.sports_team_coach_tenure.coach","p2st":"sports.sports_team_coach_tenure.from","p2en":"sports.sports_team_coach_tenure.to"}
relation_list=[spouse_dict,memofsportsteam_dict,employer_dict,coach_dict]

dataset_name="required_relations_freebase"
dataset_version="alpha-3.1"
merged_f=open(dataset_name+"_"+dataset_version+".tsv","w")
uri_prefix="http://rdf.freebase.com/ns/"

relation=relation_list[3]
for relation in relation_list:
    cnt=0
    query = '''
            SPARQL
            PREFIX ns: <http://rdf.freebase.com/ns/>
            SELECT DISTINCT ?cvt ?s ?o ?st ?en
            WHERE
            {{
                ?cvt ns:type.object.type ns:{}.
                ?cvt ns:{} ?s.
                ?cvt ns:{} ?o.

                {{
                    ?cvt ns:{} ?st.
                    OPTIONAL {{?cvt ns:{} ?en.}}
                }}
                UNION
                {{
                    ?cvt ns:{} ?en.
                    OPTIONAL {{?cvt ns:{} ?st.}}
                }}
                FILTER (?s != ?o)
            }}
            '''.format(relation["cvt_typ"],relation["p2s"],relation["p2o"],relation["p2st"],relation["p2en"],relation["p2en"],relation["p2st"])
    # print(query)
    results=query_with_odbc(query)
    print(len(results))
    # results.sort(key=lambda ele: ele[0])
    duplicate_chk_set=set()
    for each_result in results:
        line_info=[]
        
        if each_result[1].startswith(uri_prefix):
            line_info.append(each_result[1][27:])
        else:
            print("Alert!%s" %(str(each_result)))
            continue

        line_info.append(relation["r_name"])

        if each_result[2].startswith(uri_prefix):
            line_info.append(each_result[2][27:])
        else:
            print("Alert!%s" %(str(each_result)))
            continue

        if each_result[3]==None:
            line_info.append("null")
        else:
            line_info.append(each_result[3])
        if each_result[4]==None:
            line_info.append("null")
        else:
            line_info.append(each_result[4])


        dup_chk_str=str(line_info)
        if dup_chk_str in duplicate_chk_set:
            print("Duplicate line found:%s" % dup_chk_str)
            continue

        if line_info[3]!="null" and line_info[4]!="null":
            st_bc=line_info[3].startswith("-")
            st_t=line_info[3]
            en_bc=line_info[4].startswith("-")
            en_t=line_info[4]
            if ((not (st_bc and en_bc)) and st_t>en_t) or (st_bc and en_bc and st_t<en_t):
                print("Time inversion found:%s" %(str(each_result)))
        duplicate_chk_set.add(dup_chk_str)
        merged_f.write("%s\t%s\t%s\t%s\t%s\n" %(line_info[0],line_info[1],line_info[2],convert_time2value(line_info[3]),convert_time2value(line_info[4])))
        cnt+=1
    print("%s actual count %d" %(relation["r_name"],cnt))

148536
Time inversion found:('http://rdf.freebase.com/ns/m.0pdlw1b', 'http://rdf.freebase.com/ns/m.04lgg9l', 'http://rdf.freebase.com/ns/m.0pdlw1f', '1974-02-01', '1974-01-01')
Time inversion found:('http://rdf.freebase.com/ns/m.0pdlw1b', 'http://rdf.freebase.com/ns/m.0pdlw1f', 'http://rdf.freebase.com/ns/m.04lgg9l', '1974-02-01', '1974-01-01')
Time inversion found:('http://rdf.freebase.com/ns/m.011lz18v', 'http://rdf.freebase.com/ns/m.011lz18x', 'http://rdf.freebase.com/ns/m.011lytk9', '1933-04-01', '1933-01-01')
Time inversion found:('http://rdf.freebase.com/ns/m.011lz18v', 'http://rdf.freebase.com/ns/m.011lytk9', 'http://rdf.freebase.com/ns/m.011lz18x', '1933-04-01', '1933-01-01')
Time inversion found:('http://rdf.freebase.com/ns/m.0wqn57s', 'http://rdf.freebase.com/ns/m.0wqn57v', 'http://rdf.freebase.com/ns/m.0kg5hy5', '2012-02-14', '2012-01-01')
Time inversion found:('http://rdf.freebase.com/ns/m.0wqn57s', 'http://rdf.freebase.com/ns/m.0kg5hy5', 'http://rdf.freebase.com/ns/m.0wqn5

In [19]:
p_relation_list=["people.person.date_of_birth","people.deceased_person.date_of_death"]
for relation in p_relation_list:
    query = """
            SPARQL
            PREFIX ns: <http://rdf.freebase.com/ns/>
            SELECT ?s ?date
            WHERE
            {{
                ?s ns:{} ?date.
            }}
            """.format(relation)
    results=query_with_odbc(query)
    print(len(results))
    for each_result in results:
        line_info=[]
        line_info.append(each_result[0].replace(uri_prefix,""))
        line_info.append(relation)
        for i in range(3):
            line_info.append(each_result[1])
        merged_f.write("%s\t%s\t%s\t%s\t%s\n" %(line_info[0],line_info[1],line_info[2],convert_time2value(line_info[3]),convert_time2value(line_info[4])))

merged_f.close()

1924970
711838


In [21]:
query="""
SPARQL
SELECT DISTINCT ?s ?r ?otrnod
WHERE
{
    {
      ?cvt ns:type.object.type ns:people.marriage.
      ?cvt ns:people.marriage.spouse ?s.
    }
    UNION
    {
      ?cvt ns:type.object.type ns:sports.sports_team_roster.
      {?cvt ns:sports.sports_team_roster.player ?s.}
      UNION
      {?cvt ns:sports.sports_team_roster.team ?s.}
    }
    UNION
    {
      ?cvt ns:type.object.type ns:business.employment_tenure.
      {?cvt ns:business.employment_tenure.company ?s.}
      UNION
      {?cvt ns:business.employment_tenure.person ?s.}
    }
    UNION
    {
      ?cvt ns:type.object.type ns:sports.sports_team_coach_tenure.
      {?cvt ns:sports.sports_team_coach_tenure.coach ?s.}
      UNION
      {?cvt ns:sports.sports_team_coach_tenure.team ?s.}
    }
    UNION
    {
      ?s ns:people.person.date_of_birth ?date
    }
    UNION
    {
      ?s ns:people.deceased_person.date_of_death ?date
    }
    ?s ?r ?otrnod.
    ?otrnod rdfs:label ?nodlabel.
}
GROUP BY ?r
"""
results=query_with_odbc(query)

In [22]:
skipcnt=0
red_f=open("redundant_part_freebase_with_obj_type.tsv","w")
removed_r=["http://www.w3.org/1999/02/22-rdf-syntax-ns#type"]
for each_result in results:
    this_r=each_result[1].replace("http://rdf.freebase.com/ns/","")
    if this_r in removed_r:
        skipcnt+=1
        continue
    if each_result[0].startswith("http://rdf.freebase.com/ns/"):
        this_e1=each_result[0].replace("http://rdf.freebase.com/ns/","")
    else:
        print("Alert!%s\n" %(str(each_result)))
    if each_result[2].startswith("http://rdf.freebase.com/ns/"):
        this_e2=each_result[2].replace("http://rdf.freebase.com/ns/","")
    else:
        print("Alert!%s\n" %(str(each_result)))
    
    red_f.write("%s\t%s\t%s\tnull\tnull\n" %(this_e1,this_r,this_e2))

red_f.close()

In [23]:
final_file=open("all_relations_with_redundant_freebase_alpha-1.1.tsv","w")
merge_file_list=["required_relations_freebase_alpha-3.1.tsv","redundant_part_freebase_with_obj_type.tsv"]
for file_name in merge_file_list:
    now_file=open(file_name,"r")
    file_content=now_file.readlines()
    for each_line in file_content:
        final_file.write(each_line)
    now_file.close()
final_file.close()

In [12]:
# results.pop(0)
# results.pop(0)
# print(results)
redundant_relation=[]
for each_res in results:
    redundant_relation.append(each_res[0].replace("http://rdf.freebase.com/ns/",""))
log_file=open("freebase_logfile.txt","w")
for each_r in redundant_relation:
    cnt=0
    query="""
        SPARQL
        SELECT DISTINCT ?e1 ?e2
        {{
            ?e1 ns:{} ?e2.
            ?e1 rdfs:label ?e1label.
            ?e2 rdfs:label ?e2label.
        }}
    """.format(each_r)
    results=query_with_odbc(query)
    print("%s complete!" %each_r)
    duplicate_chk_set=set()
    for each_result in results:
        line_info=[]
        
        if each_result[0].startswith(uri_prefix):
            line_info.append(each_result[0][27:])
        else:
            log_file.write("Alert!IN %s:%s\n" %(each_r,str(each_result)))
            continue

        line_info.append(each_r)

        if each_result[1].startswith(uri_prefix):
            line_info.append(each_result[1][27:])
        else:
            log_file.write("Alert!IN %s:%s\n" %(each_r,str(each_result)))
            continue

        dup_chk_str=str(line_info)
        if dup_chk_str in duplicate_chk_set:
            print("Duplicate line found:%s" % dup_chk_str)
            continue

        duplicate_chk_set.add(dup_chk_str)
        merged_f.write("%s\t%s\t%s\tnull\tnull\n" %(line_info[0],line_info[1],line_info[2]))
        cnt+=1
    print("%s(redundant) actual count %d" %(each_r,cnt))
merged_f.close()
log_file.close()


common.topic.notable_types complete!


ValueError: I/O operation on closed file.

In [19]:
test_str="1244 00:00:00-04:00"
print(convert_time2value(test_str))

12440101


In [7]:
#get SPARQL
relation=relation_list[0]
query = '''
            SPARQL
            PREFIX ns: <http://rdf.freebase.com/ns/>
            SELECT DISTINCT ?cvt ?s ?o ?st ?en
            WHERE
            {{
                ?cvt ns:type.object.type ns:{}.
                ?cvt ns:{} ?s.
                ?cvt ns:{} ?o.

                {{
                    ?cvt ns:{} ?st.
                    OPTIONAL {{?cvt ns:{} ?en.}}
                }}
                UNION
                {{
                    ?cvt ns:{} ?en.
                    OPTIONAL {{?cvt ns:{} ?st.}}
                }}
                FILTER (?s != ?o)
            }}
            '''.format(relation["cvt_typ"],relation["p2s"],relation["p2o"],relation["p2st"],relation["p2en"],relation["p2en"],relation["p2st"])
print(query)


            SPARQL
            PREFIX ns: <http://rdf.freebase.com/ns/>
            SELECT DISTINCT ?cvt ?s ?o ?st ?en
            WHERE
            {
                ?cvt ns:type.object.type ns:people.marriage.
                ?cvt ns:people.marriage.spouse ?s.
                ?cvt ns:people.marriage.spouse ?o.

                {
                    ?cvt ns:people.marriage.from ?st.
                    OPTIONAL {?cvt ns:people.marriage.to ?en.}
                }
                UNION
                {
                    ?cvt ns:people.marriage.to ?en.
                    OPTIONAL {?cvt ns:people.marriage.from ?st.}
                }
                FILTER (?s != ?o)
            }
            


In [2]:
query="""
SPARQL
PREFIX ns: <http://rdf.freebase.com/ns/>
SELECT DISTINCT ?cvt  ?st ?en
WHERE
{
    ?cvt ns:type.object.type ns:people.marriage.
    ?cvt ns:people.marriage.spouse ns:m.02ctvl.
    ?cvt ns:people.marriage.spouse ns:m.018wsf.

    {
        ?cvt ns:people.marriage.from ?st.
        OPTIONAL {?cvt ns:people.marriage.to ?en.}
    }
    UNION
    {
        ?cvt ns:people.marriage.to ?en.
        OPTIONAL {?cvt ns:people.marriage.from ?st.}
    }

}
"""
results=query_with_odbc(query)
print(results)


[('http://rdf.freebase.com/ns/m.05st71t', '0000-01-01', '0004-01-01')]


In [4]:
convert_time2value('0000-01-01')

'00000101'